In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itbr"
fs_method, fs_ratio = "svc", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9130 | AUPR: 0.6407 | Acc: 95.43%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9184 | AUPR: 0.6574 | Acc: 95.51%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9228 | AUPR: 0.6744 | Acc: 95.52%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9149 | AUPR: 0.6468 | Acc: 95.46%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9203 | AUPR: 0.6653 | Acc: 95.49%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9221 | AUPR: 0.6737 | Acc: 95.49%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9228 | AUPR: 0.6744 | Acc: 95.52%
Internal test: AUC: 0.9286 | AUPR: 0.6980 | Acc: 95.06%
External test: AUC: 0.9194 | AUPR: 0.6661 | Acc: 95.30%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9159 | AUPR: 0.6215 | Acc: 94.97%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9197 | AUPR: 0.6300 | Acc: 95.08%
model(kernel=linear, C=1)
Valid:         AUC: 0.9199 | AUPR: 0.6280 | Acc: 95.06%
best model(kernel=linear, C=1)
Valid:         AUC: 0.9199 | AUPR: 0.6280 | Acc: 95.06%
Internal test: AUC: 0.9257 | AUPR: 0.6636 | Acc: 94.91%
External test: AUC: 0.9142 | AUPR: 0.6010 | Acc: 94.54%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7499 | AUPR: 0.4168 | Acc: 94.73%
model(n_neighbors=5)
Valid:         AUC: 0.7774 | AUPR: 0.4612 | Acc: 94.93%
model(n_neighbors=7)
Valid:         AUC: 0.7963 | AUPR: 0.4858 | Acc: 95.08%
best model(n_neighbors=7)
Valid:         AUC: 0.7963 | AUPR: 0.4858 | Acc: 95.08%
Internal test: AUC: 0.8111 | AUPR: 0.5083 | Acc: 94.79%
External test: AUC: 0.7984 | AUPR: 0.4831 | Acc: 94.67%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8852 | AUPR: 0.4819 | Acc: 86.28%
best model()
Valid:         AUC: 0.8852 | AUPR: 0.4819 | Acc: 86.28%
Internal test: AUC: 0.8983 | AUPR: 0.5336 | Acc: 88.76%
External test: AUC: 0.8872 | AUPR: 0.4804 | Acc: 87.62%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9198 | AUPR: 0.6545 | Acc: 95.47%
model(C=10.0)
Valid:         AUC: 0.9204 | AUPR: 0.6551 | Acc: 95.43%
model(C=100.0)
Valid:         AUC: 0.9206 | AUPR: 0.6555 | Acc: 95.39%
best model(C=100.0)
Valid:         AUC: 0.9206 | AUPR: 0.6555 | Acc: 95.39%
Internal test: AUC: 0.9287 | AUPR: 0.6764 | Acc: 95.28%
External test: AUC: 0.9146 | AUPR: 0.6212 | Acc: 94.94%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8755 | AUPR: 0.5615 | Acc: 85.38%
model(criterion=log_loss)
Valid:         AUC: 0.8779 | AUPR: 0.5724 | Acc: 84.65%
model(criterion=entropy)
Valid:         AUC: 0.8779 | AUPR: 0.5724 | Acc: 84.65%
best model(criterion=log_loss)
Valid:         AUC: 0.8779 | AUPR: 0.5724 | Acc: 84.65%
Internal test: AUC: 0.8823 | AUPR: 0.6033 | Acc: 88.91%
External test: AUC: 0.8744 | AUPR: 0.5693 | Acc: 88.15%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9174 | AUPR: 0.6603 | Acc: 89.44%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9171 | AUPR: 0.6595 | Acc: 89.67%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9174 | AUPR: 0.6600 | Acc: 89.76%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9168 | AUPR: 0.6606 | Acc: 89.43%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9171 | AUPR: 0.6610 | Acc: 89.75%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9170 | AUPR: 0.6611 | Acc: 89.81%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9168 | AUPR: 0.6606 | Acc: 89.43%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9171 | AUPR: 0.6610 | Acc: 89.75%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9170 | AUPR: 0.6611 | Acc: 89.81%
best model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9174 | AUPR: 0.6603 | Acc: 89

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9204 | AUPR: 0.6527 | Acc: 95.10%
model(criterion=squared_error)
Valid:         AUC: 0.9201 | AUPR: 0.6413 | Acc: 95.02%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9204 | AUPR: 0.6527 | Acc: 95.10%
Internal test: AUC: 0.9247 | AUPR: 0.6833 | Acc: 95.30%
External test: AUC: 0.9182 | AUPR: 0.6504 | Acc: 94.99%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9141 | AUPR: 0.6356 | Acc: 95.27%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9160 | AUPR: 0.6425 | Acc: 95.30%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9162 | AUPR: 0.6443 | Acc: 95.31%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9162 | AUPR: 0.6443 | Acc: 95.31%
Internal test: AUC: 0.9191 | AUPR: 0.6794 | Acc: 95.30%
External test: AUC: 0.9110 | AUPR: 0.6314 | Acc: 94.98%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)